In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load the dataset
data = pd.read_csv('predictionChallenge.csv')

In [2]:
#!curl 'https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD' -o restaurants.csv


In [3]:
#restdf = pd.read_csv('restaurants.csv')


In [4]:
# Filter out rows with NaN or 0 latitude or longitude
#restdf = restdf[(restdf['Latitude'].notna()) & (restdf['Latitude'] != 0) &
     #           (restdf['Longitude'].notna()) & (restdf['Longitude'] != 0)]

# Save the filtered DataFrame back to a CSV file
#restdf.to_csv('filtered_restaurants.csv', index=False)
#restdf1 = pd.read_csv('filtered_restaurants.csv')
#restdf1

In [5]:
###
#import numpy as np

#latitudes_airbnb = data['latitude'].values[:, np.newaxis]  # Convert to 2D array
#longitudes_airbnb = data['longitude'].values[:, np.newaxis]  # Convert to 2D array
#latitudes_restaurant = restdf1['Latitude'].values
#longitudes_restaurant = restdf1['Longitude'].values

# Calculate distances using vectorized operations
#distances = np.sqrt((latitudes_airbnb - latitudes_restaurant) ** 2 + (longitudes_airbnb - longitudes_restaurant) ** 2)

# Find the closest restaurant for each Airbnb
#closest_restaurant_indices = np.argmin(distances, axis=1)
#closest_restaurant_distances = distances[np.arange(len(distances)), closest_restaurant_indices]
#closest_restaurant_names = restdf1.iloc[closest_restaurant_indices]['Restaurant Name'].values

# Add the closest restaurant distance and name to the Airbnb data
#data['distance_to_nearest_restaurant'] = closest_restaurant_distances
#data['closest_restaurant_name'] = closest_restaurant_names

# Count the number of restaurants within a certain distance from each Airbnb
#threshold_distance = 0.5  # Define the threshold distance in kilometers
#num_restaurants_within_distance = np.sum(distances <= threshold_distance, axis=1)

# Add the number of restaurants within the threshold distance to the Airbnb data
#data['num_restaurants_within_distance'] = num_restaurants_within_distance

# Save the updated DataFrame to the existing predictionChallenge.csv file
#data.to_csv('predictionChallenge.csv', mode='a', header=False, index=False)
###

In [ ]:
data['duration_of_service_months'] = data['number_of_reviews'] / data['reviews_per_month']

# Calculate price per minimum nights
data['price_per_minimum_nights'] = data['Price($)'] / data['minimum nights']

data.to_csv('predictionChallenge.csv', index=False)

In [6]:
neighborhood_avg_price = data.groupby('neighbourhood')['Price($)'].mean()

def calculate_price_competitiveness(row):
    neighborhood = row['neighbourhood']
    price = row['Price($)']
    avg_price = neighborhood_avg_price.get(neighborhood, 0)  # Use 0 as default if neighborhood not found
    if avg_price == 0:
        return 'Unknown'
    elif price < avg_price:
        return 'Competitive'
    elif price > avg_price:
        return 'Not Competitive'
    else:
        return 'Equal'

data['price_competitiveness'] = data.apply(calculate_price_competitiveness, axis=1)

data.to_csv('predictionChallenge.csv', index=False)

In [26]:
# Load the first Airbnb entry
airbnb_df = pd.read_csv('predictionChallenge.csv')
first_airbnb = airbnb_df.iloc[[5]]

# Load subway station data
subway_df = pd.read_csv('subway.csv')

# Define walking distance threshold in miles
walking_distance_threshold = 0.1  # Example threshold (adjust as needed)

# Calculate distance between the first Airbnb listing and each subway station
def calculate_distance(lat1, lon1, lat2, lon2):
    # Haversine formula to calculate distance between two points on Earth's surface
    R = 3958.8  # Earth radius in miles
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Count nearby subway stations for the first Airbnb listing
nearby_station_count = 0
for _, subway_row in subway_df.iterrows():
    distance = calculate_distance(first_airbnb['latitude'].values[0], first_airbnb['longitude'].values[0],
                                   subway_row['Station Latitude'], subway_row['Station Longitude'])
    if distance <= walking_distance_threshold:
        nearby_station_count += 1

# Output the count of nearby subway stations
print("Number of nearby subway stations for the first Airbnb listing:", nearby_station_count)

Number of nearby subway stations for the first Airbnb listing: 0


In [29]:
# Load the first Airbnb entry
airbnb_df = pd.read_csv('predictionChallenge.csv')
first_airbnb = airbnb_df.iloc[[6]]

# Load subway station data
subway_df = pd.read_csv('subway.csv')

# Calculate distance between the first Airbnb listing and each subway station
def calculate_distance(lat1, lon1, lat2, lon2):
    # Haversine formula to calculate distance between two points on Earth's surface
    R = 3958.8  # Earth radius in miles
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Find distance to the nearest subway station for the first Airbnb listing
nearest_station_distance = float('inf')
for _, subway_row in subway_df.iterrows():
    distance = calculate_distance(first_airbnb['latitude'].values[0], first_airbnb['longitude'].values[0],
                                   subway_row['Station Latitude'], subway_row['Station Longitude'])
    if distance < nearest_station_distance:
        nearest_station_distance = distance

# Output the distance to the nearest subway station
print("Distance to the nearest subway station for the first Airbnb listing:", nearest_station_distance, "miles")


Distance to the nearest subway station for the first Airbnb listing: 0.2287791254868868 miles


In [48]:
import pandas as pd
import numpy as np

# Load Airbnb and subway station data
airbnb_df = pd.read_csv('predictionChallenge.csv')
subway_df = pd.read_csv('subway.csv')

# Calculate distance between two points using 2D formula and convert to miles
def calculate_distance(lat1, lon1, lat2, lon2):
    # Calculate differences in coordinates
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    
    # Calculate distance using 2D formula
    distance = np.sqrt(delta_lat**2 + delta_lon**2)
    
    # Convert distance to miles
    distance_miles = distance * 69  # 1 degree of latitude is approximately 69 miles
    return distance_miles

# Define the threshold distance in miles
threshold_distance_miles = 0.5

# Get the latitude and longitude of the first Airbnb entry
first_airbnb = airbnb_df.iloc[0]
first_lat = first_airbnb['latitude']
first_lon = first_airbnb['longitude']

# Initialize a variable to store the count of nearby subway stations for the first Airbnb listing
nearby_station_count = 0

# Loop over each subway station
for _, subway_row in subway_df.iterrows():
    # Calculate the distance between the first Airbnb listing and the current subway station
    distance_miles = calculate_distance(first_lat, first_lon, subway_row['Entrance Latitude'], subway_row['Entrance Longitude'])
    
    # If the distance is within the threshold distance, increment the count
    if distance_miles <= threshold_distance_miles:
        nearby_station_count += 1

# Print the count of subway stations within 0.5 miles for the first Airbnb entry
print("Count of subway stations within 0.5 miles for the first Airbnb entry:", nearby_station_count)

Count of subway stations within 0.5 miles for the first Airbnb entry: 4


In [62]:
# Load Airbnb and subway station data
airbnb_df = pd.read_csv('predictionChallenge.csv')
subway_df = pd.read_csv('subway.csv')

# Calculate distance between two points using 2D formula and convert to miles
def calculate_distance(lat1, lon1, lat2, lon2):
    # Calculate differences in coordinates
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    
    # Calculate distance using 2D formula
    distance = np.sqrt(delta_lat**2 + delta_lon**2)
    
    # Convert distance to miles
    distance_miles = distance * 69  # 1 degree of latitude is approximately 69 miles
    return distance_miles

# Define the threshold distance in miles
threshold_distance_miles = 0.5

# Initialize a list to store the counts of nearby subway stations for each Airbnb entry
subway_station_counts = []

# Loop over each Airbnb entry
for _, airbnb_row in airbnb_df.iterrows():
    # Get the latitude and longitude of the Airbnb entry
    airbnb_lat = airbnb_row['latitude']
    airbnb_lon = airbnb_row['longitude']
    
    # Initialize a variable to store the count of nearby subway stations for the current Airbnb entry
    nearby_station_count = 0
    
    # Loop over each subway station
    for _, subway_row in subway_df.iterrows():
        # Calculate the distance between the Airbnb entry and the current subway station
        distance_miles = calculate_distance(airbnb_lat, airbnb_lon, subway_row['Entrance Latitude'], subway_row['Entrance Longitude'])
        
        # If the distance is within the threshold distance, increment the count
        if distance_miles <= threshold_distance_miles:
            nearby_station_count += 1
    
    # Add the count of nearby subway stations to the list
    subway_station_counts.append(nearby_station_count)

# Add the list of counts as a new column in the Airbnb DataFrame
airbnb_df['subway_station_count'] = subway_station_counts

# Save the modified DataFrame back to the predictionChallenge.csv file
airbnb_df.to_csv('predictionChallenge.csv', index=False)

In [65]:
import pandas as pd

# Read the first CSV file
df1 = pd.read_csv('predictionChallenge.csv')

# Read the second CSV file
df2 = pd.read_csv('your_dataset_with_row_numbers.csv')

# Merge the DataFrames based on a common column
merged_df = pd.merge(df1, df2, on='ID', how='inner')  # Change 'common_column' to the column name you want to merge on

# Select the column from df2 that you want to add to df1
column_to_add = merged_df['neighbourhood group']  # Change 'column_to_add' to the column name you want to add

# Add the selected column to df1
df1['neighbourhood group'] = column_to_add  # Change 'new_column' to the name you want for the added column

# Write the updated DataFrame to a new CSV file
df1.to_csv('predictionChallenge1.csv', index=False)

In [71]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('predictionChallenge1.csv')

# Feature Engineering
# Define complex conditions for categorizing likelihood of booking
conditions = [
    ((df['minimum nights'] < 5) &  # Short minimum nights
     (df['number_of_reviews'] > 20) &  # High number of reviews
     (df['reviews_per_month'] > 0.5) &  # High reviews per month
     (df['floor'] < 5) &  # Low floor
     (df['noise(dB)'] < 50) &  # Low noise level
     (df['Price($)'] < 200) &  # Low price
     (df['mean_review_score'] > 4) &  # High mean review scores
     (df['Deal Quality'] == 'Good') &  # Good deal quality
     (df['distance_to_nearest_attraction'] < 500) &  # Nearby attractions
     (df['subway_station_count'] > 5) &  # Many nearby subway stations
     (df['neighbourhood group'] == 'Manhattan')),  # Specific neighbourhood group
    ((df['minimum nights'] < 5) &  
     (df['number_of_reviews'] > 20) &  
     (df['reviews_per_month'] > 0.5) &  
     (df['floor'] < 5) &  
     (df['noise(dB)'] < 50) &  
     (df['Price($)'] < 200) &  
     (df['mean_review_score'] > 4) &  
     (df['Deal Quality'] == 'Good') &  
     (df['distance_to_nearest_attraction'] < 500) &  
     (df['subway_station_count'] > 5) &  
     (df['neighbourhood group'] == 'Brooklyn')),  
    # Add conditions for other neighbourhood groups
    ((df['minimum nights'] < 5) &  
     (df['number_of_reviews'] > 20) &  
     (df['reviews_per_month'] > 0.5) &  
     (df['floor'] < 5) &  
     (df['noise(dB)'] < 50) &  
     (df['Price($)'] < 200) &  
     (df['mean_review_score'] > 4) &  
     (df['Deal Quality'] == 'Good') &  
     (df['distance_to_nearest_attraction'] < 500) &  
     (df['subway_station_count'] > 5) &  
     (df['neighbourhood group'] == 'Queens')),  
    ((df['minimum nights'] < 5) &  
     (df['number_of_reviews'] > 20) &  
     (df['reviews_per_month'] > 0.5) &  
     (df['floor'] < 5) &  
     (df['noise(dB)'] < 50) &  
     (df['Price($)'] < 200) &  
     (df['mean_review_score'] > 4) &  
     (df['Deal Quality'] == 'Good') &  
     (df['distance_to_nearest_attraction'] < 500) &  
     (df['subway_station_count'] > 5) &  
     (df['neighbourhood group'] == 'Bronx'))
]

# Define corresponding categories
categories = ['Unlikely', 'Likely', 'Highly Likely', 'Other']

# Create a new column 'booking_likelihood' based on the conditions
df['booking_likelihood'] = pd.cut(df['number_of_reviews'], bins=len(conditions), labels=categories)

# Save the modified DataFrame to a new CSV file
df.to_csv('modified_data.csv', index=False)